In [5]:
import numpy as np
import pandas as pd
import praw
import time

from my_praw_info import get_client_id, get_client_secret, get_user_agent

# create a Reddit read-only instance
reddit = praw.Reddit(client_id=get_client_id(),
                    client_secret=get_client_secret(),
                    user_agent=get_user_agent())

In [6]:
from subreddit_list import subreddit_list

subreddits = subreddit_list()

In [61]:
# merge BPD test post csv's
# COMPLETED MAY 22

#subreddit = 'BPD'
#filepath = '../data/posts/'+subreddit+'.csv'
#merge_index = '../data/posts/test.csv'

#post_df = pd.concat(map(pd.read_csv,[filepath,merge_index]), 
#                    ignore_index = True)

#post_df.to_csv(filepath)

In [62]:
len(post_df.index)

1750

In [33]:
cleaned_df = post_df.drop_duplicates(
    subset = ['ID'],
    keep = 'last').reset_index(drop = True)

cleaned_df = cleaned_df[cleaned_df.Title != 'Title']

In [39]:
len(cleaned_df)

1042

In [35]:
cleaned_df.sort_values(by=['ID']).head()
len(cleaned_df[cleaned_df.Title == 'Title'])

0

In [60]:
filepath = '../data/comments/BPD_comments.csv'
comment_df = pd.read_csv(filepath)

# drop duplicates by post index
cleaned_df = comment_df.drop_duplicates(
subset = ['ID','Comment'],
keep = 'last').reset_index(drop = True)

print(len(comment_df)-len(cleaned_df),'duplicates dropped from',filepath)

# remove duplicate of column headers
print(len(cleaned_df[cleaned_df.Comment == 'Comment']))

66 duplicates dropped from ../data/comments/BPD_comments.csv
0


In [4]:
# create empty dictionary for subreddit dataframes
post_dfs = dict.fromkeys(subreddits)

# fill dictionary with dataframes
for subreddit in subreddits:
    post_dfs[subreddit]= pd.read_csv('data/posts/'+subreddit+'.csv')
    print('r/'+subreddit,'posts dataframe has length',len(post_dfs[subreddit]))

r/BPD posts dataframe has length 1000
r/BorderlinePDisorder posts dataframe has length 997
r/BPDmemes posts dataframe has length 989
r/SuicideWatch posts dataframe has length 990
r/QuietBPD posts dataframe has length 9
r/BPDrecovery posts dataframe has length 965
r/mentalhealth posts dataframe has length 996
r/BPDsupport posts dataframe has length 983


In [5]:
post_dfs['BorderlinePDisorder'].sample(2)

,Title,Post Date,Post Text,Author,Author Flair,Original Content,ID,Score,Total Comments,Post URL
592,Youngesters with BPD,1.714637e+09,Any young people here who have diagnosed BPD? ...,dumbcat664,NaN,False,1cia3e1,5,11,https://www.reddit.com/r/BorderlinePDisorder/c...
127,Dont know what to do about my therapist,1.715865e+09,Ive been seeing a therapist the past month so ...,OkCaterpillar3483,NaN,False,1ctci06,0,2,https://www.reddit.com/r/BorderlinePDisorder/c...


In [6]:
length = 0

for subreddit in subreddits:
    length += len(post_dfs[subreddit])

print(length,'total posts')

6929 total posts


In [120]:
# create empty dictionary for subreddit dataframes
comment_dfs = dict.fromkeys(subreddits)

# fill dictionary with dataframes
for subreddit in subreddits:
    comment_dfs[subreddit]= pd.read_csv('../data/comments/'+subreddit+'_comments.csv')
    print('r/'+subreddit,'comments dataframe has length',len(comment_dfs[subreddit]))

r/BPD comments dataframe has length 7418
r/BorderlinePDisorder comments dataframe has length 9176
r/BPDmemes comments dataframe has length 8417
r/SuicideWatch comments dataframe has length 5349
r/QuietBPD comments dataframe has length 44
r/BPDrecovery comments dataframe has length 4936
r/mentalhealth comments dataframe has length 4539
r/BPDsupport comments dataframe has length 2718


In [8]:
comment_dfs['BorderlinePDisorder'].sample(2)

,ID,Comment,Author,OP,Post Date
5303,1cheqpx,I knew I wasn’t supposed to do it(I know I sho...,PastaMakerFullOfBean,False,1.714592e+09
7682,1cak101,Inspiring✨,ivee7,True,1.713880e+09


In [124]:
type(comment_dfs['BPD']['OP'][0])

numpy.bool_

In [11]:
length = 0

for subreddit in subreddits:
    length += len(comment_dfs[subreddit])

print(length,'total comments')

36565 total comments


### Developing a merge function for .new posts and comments

Goal:

* Develop a function that creates a dataframe for a single post -- post vs. comments
    * Headers : ID (str) | Title (str) | Text (str) | Author (str) | OP (int) | Is_Post (int) | Post Date (?) | Subreddit (str)

In [41]:
import numpy as np
import pandas as pd
import praw
import time

from subreddit_list import subreddit_list

subreddits = subreddit_list()

# create empty dictionary for subreddit dataframes
post_dfs = dict.fromkeys(subreddits)

# fill dictionary with dataframes
for subreddit in subreddits:
    post_dfs[subreddit]= pd.read_csv('../data/posts/'+subreddit+'.csv')

post_dfs['BPD']

,Title,Post Date,Post Text,Author,Author Flair,Original Content,ID,Score,Total Comments,Post URL
0,Should I go to the hospital,1716145973,Hi\nIm 15f and when I was 12 i got sent to the...,a_lost_poet,NaN,False,1cvvu64,1,3,https://www.reddit.com/r/BPD/comments/1cvvu64/...
1,manic state after breakup (F 20) smoking a ton...,1716145641,I just went through a breakup with my boyfrien...,VastIntelligent569,NaN,False,1cvvpqr,1,3,https://www.reddit.com/r/BPD/comments/1cvvpqr/...
2,Why,1716145558,Why do I only like people who do not like me? ...,MiserableSweet4033,NaN,False,1cvvom7,1,1,https://www.reddit.com/r/BPD/comments/1cvvom7/...
3,Will she come back?,1716145275,Will she come back?\n\nThis is my first ever r...,Opposite-Nail-273,NaN,False,1cvvkpe,1,2,https://www.reddit.com/r/BPD/comments/1cvvkpe/...
4,Guilt,1716144789,I’ve been missing 2 weeks straight of classes ...,Spiritual-Video4481,NaN,False,1cvvdvx,1,2,https://www.reddit.com/r/BPD/comments/1cvvdvx/...
...,...,...,...,...,...,...,...,...,...,...
1130,Social media worsens Identity Disturbance - Ge...,1715828159,I was reflecting on my thoughts and behavior t...,Mother-Focus-84,NaN,False,1ct32jo,6,0,https://www.reddit.com/r/BPD/comments/1ct32jo/...
1131,How is listening to people complain about thei...,1715826428,I fucking hate DBT group. How is listening to ...,StreetAd7840,NaN,False,1ct2jbu,117,79,https://www.reddit.com/r/BPD/comments/1ct2jbu/...
1132,is my bestfriend avoiding me or am i being par...,1715825337,I (f18) moved to another country with my bestf...,Outrageous_Voice1506,NaN,False,1ct26nb,1,0,https://www.reddit.com/r/BPD/comments/1ct26nb/...
1133,what do you do when you have no memory of what...,1715824490,9/10 times when i fight with my partner and it...,Ok_Salamander_4000,NaN,False,1ct1whn,3,1,https://www.reddit.com/r/BPD/comments/1ct1whn/...


In [97]:
merge_dfs = {'ID':[], 'Title':[], 'Text':[], 'Author':[], 'OP':[], 'Is Post':[], 'Post Date':[], 'Subreddit':[]}

post_dfs['BPD']

len(post_dfs['BPD'])

1135

In [98]:
for subreddit in subreddits:
    for index, row in post_dfs[subreddit].iterrows():
        merge_dfs['ID'].append(row['ID'])
        merge_dfs['Title'].append(row['Title'])
        merge_dfs['Text'].append(row['Post Text'])
        merge_dfs['Author'].append(row['Author'])
        merge_dfs['OP'].append(int(1))
        merge_dfs['Is Post'].append(int(1))
        merge_dfs['Post Date'].append(row['Post Date'])
        merge_dfs['Subreddit'].append(subreddit)

post_dict = merge_dfs.copy()

merge_dfs = pd.DataFrame(merge_dfs)
merge_dfs

,ID,Title,Text,Author,OP,Is Post,Post Date,Subreddit
0,1cvvu64,Should I go to the hospital,Hi\nIm 15f and when I was 12 i got sent to the...,a_lost_poet,1,1,1.716146e+09,BPD
1,1cvvpqr,manic state after breakup (F 20) smoking a ton...,I just went through a breakup with my boyfrien...,VastIntelligent569,1,1,1.716146e+09,BPD
2,1cvvom7,Why,Why do I only like people who do not like me? ...,MiserableSweet4033,1,1,1.716146e+09,BPD
3,1cvvkpe,Will she come back?,Will she come back?\n\nThis is my first ever r...,Opposite-Nail-273,1,1,1.716145e+09,BPD
4,1cvvdvx,Guilt,I’ve been missing 2 weeks straight of classes ...,Spiritual-Video4481,1,1,1.716145e+09,BPD
...,...,...,...,...,...,...,...,...
9052,pbfh2u,We´ve made a new Video trying to explain how B...,Hope you guys like it \n\n&#x200B;\n\n[https:/...,BorderlineHR,1,1,1.629911e+09,BPDsupport
9053,p2g02n,A useful video about Dissociation and How to d...,We have made a new Video explaining what Disso...,BorderlineHR,1,1,1.628698e+09,BPDsupport
9054,ox2vtz,Male Who Has BPD (NEWLY DIAGNOSED). Are any of...,"When I'm in a high mood, whether I'm happy or ...",RecommendationWild71,1,1,1.627995e+09,BPDsupport
9055,ote479,"Hi guys, here´s a new video from us detailing ...",NaN,BorderlineHR,1,1,1.627492e+09,BPDsupport


#### Merge comments with posts

In [110]:
if (merge_dfs.loc[(merge_dfs['ID'] == '1cvvu64')#row['ID']) 
              & (merge_dfs['Is Post'] == 1)]['Author'][0] == 'a_lost_poet'):
    print('tis true')

# (merge_dfs['ID'] == row['ID'])

#merge_dfs['Author'].astype('string')
# merge_dfs.info()

merge_dfs.info()

tis true
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9057 entries, 0 to 9056
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         9057 non-null   object 
 1   Title      9057 non-null   object 
 2   Text       7822 non-null   object 
 3   Author     8805 non-null   object 
 4   OP         9057 non-null   int64  
 5   Is Post    9057 non-null   int64  
 6   Post Date  9057 non-null   float64
 7   Subreddit  9057 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 566.2+ KB


In [128]:
merge_dfs = pd.read_csv('../data/merged/new_posts_merged.csv')
len(merge_dfs)

9057

In [203]:
merge_dfs

,ID,Title,Text,Author,OP,Is Post,Post Date,Subreddit
0,1cvvu64,Should I go to the hospital,Hi\nIm 15f and when I was 12 i got sent to the...,a_lost_poet,1,1,1.716146e+09,BPD
1,1cvvpqr,manic state after breakup (F 20) smoking a ton...,I just went through a breakup with my boyfrien...,VastIntelligent569,1,1,1.716146e+09,BPD
2,1cvvom7,Why,Why do I only like people who do not like me? ...,MiserableSweet4033,1,1,1.716146e+09,BPD
3,1cvvkpe,Will she come back?,Will she come back?\n\nThis is my first ever r...,Opposite-Nail-273,1,1,1.716145e+09,BPD
4,1cvvdvx,Guilt,I’ve been missing 2 weeks straight of classes ...,Spiritual-Video4481,1,1,1.716145e+09,BPD
...,...,...,...,...,...,...,...,...
9052,pbfh2u,We´ve made a new Video trying to explain how B...,Hope you guys like it \n\n&#x200B;\n\n[https:/...,BorderlineHR,1,1,1.629911e+09,BPDsupport
9053,p2g02n,A useful video about Dissociation and How to d...,We have made a new Video explaining what Disso...,BorderlineHR,1,1,1.628698e+09,BPDsupport
9054,ox2vtz,Male Who Has BPD (NEWLY DIAGNOSED). Are any of...,"When I'm in a high mood, whether I'm happy or ...",RecommendationWild71,1,1,1.627995e+09,BPDsupport
9055,ote479,"Hi guys, here´s a new video from us detailing ...",NaN,BorderlineHR,1,1,1.627492e+09,BPDsupport


In [205]:
from clean_post_csv import clean_post_csv 
clean_post_csv('../data/merged/new_posts_merged.csv')

66 duplicates dropped from ../data/merged/new_posts_merged.csv


In [290]:
# load post dataframe
merge_dfs = pd.read_csv('../data/merged/new_posts_merged.csv',index_col=False)

list = ['ID', 'Title', 'Text', 'Author','Subreddit']
for item in list:
    merge_dfs[item] = merge_dfs[item].astype('string')

merge_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8991 entries, 0 to 8990
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         8991 non-null   string 
 1   Title      8991 non-null   string 
 2   Text       7756 non-null   string 
 3   Author     8739 non-null   string 
 4   OP         8991 non-null   int64  
 5   Is Post    8991 non-null   int64  
 6   Post Date  8991 non-null   float64
 7   Subreddit  8991 non-null   string 
dtypes: float64(1), int64(2), string(5)
memory usage: 562.1 KB


In [296]:
#merge_dfs.loc[merge_dfs['ID'] == '1cvvom7']['Title'].values[0]

merge_dfs['Title']#.loc[merge_dfs['Title'] == 'Why']
merge_dfs[merge_dfs.duplicated('Is Post',keep=False)==True]#.sort_values('Title')


,ID,Title,Text,Author,OP,Is Post,Post Date,Subreddit
0,1cvvu64,Should I go to the hospital,Hi Im 15f and when I was 12 i got sent to the ...,a_lost_poet,1,1,1.716146e+09,BPD
1,1cvvpqr,manic state after breakup (F 20) smoking a ton...,I just went through a breakup with my boyfrien...,VastIntelligent569,1,1,1.716146e+09,BPD
2,1cvvom7,Why,Why do I only like people who do not like me? ...,MiserableSweet4033,1,1,1.716146e+09,BPD
3,1cvvkpe,Will she come back?,Will she come back? This is my first ever red...,Opposite-Nail-273,1,1,1.716145e+09,BPD
4,1cvvdvx,Guilt,I’ve been missing 2 weeks straight of classes ...,Spiritual-Video4481,1,1,1.716145e+09,BPD
...,...,...,...,...,...,...,...,...
8986,pbfh2u,We´ve made a new Video trying to explain how B...,Hope you guys like it &#x200B; [https://www...,BorderlineHR,1,1,1.629911e+09,BPDsupport
8987,p2g02n,A useful video about Dissociation and How to d...,We have made a new Video explaining what Disso...,BorderlineHR,1,1,1.628698e+09,BPDsupport
8988,ox2vtz,Male Who Has BPD (NEWLY DIAGNOSED). Are any of...,"When I'm in a high mood, whether I'm happy or ...",RecommendationWild71,1,1,1.627995e+09,BPDsupport
8989,ote479,"Hi guys, here´s a new video from us detailing ...",<NA>,BorderlineHR,1,1,1.627492e+09,BPDsupport


In [320]:
import numpy as np
import pandas as pd
import praw
import time

from subreddit_list import subreddit_list

subreddits = subreddit_list()

# load post dataframe
merge_dfs = pd.read_csv('../data/merged/new_posts_merged.csv',index_col=False)

# correct column data types
list = ['ID', 'Title', 'Text', 'Author','Subreddit']
for item in list:
    merge_dfs[item] = merge_dfs[item].astype('string')

# create empty dictionary for subreddit dataframes
comment_dfs = dict.fromkeys(subreddits)

# fill dictionary with dataframes
for subreddit in subreddits:
    comment_dfs[subreddit]= pd.read_csv('../data/comments/'+subreddit+'_comments.csv')

# append comments to merged post dataframe
for subreddit in subreddits:
    reduced_comment_df = {'ID':[], 'Title':[], 'Text':[], 
                      'Author':[], 'OP':[], 'Is Post':[], 
                      'Post Date':[], 'Subreddit':[]}
    for index, row in comment_dfs[subreddit].iterrows():
        reduced_comment_df['ID'].append(row['ID'])
        reduced_comment_df['Title'].append(merge_dfs.loc[(merge_dfs['ID'] == row['ID']) 
            & (merge_dfs['Is Post'] == 1)]['Title'].item())
        reduced_comment_df['Text'].append(row['Comment'])
        reduced_comment_df['Author'].append(row['Author'])
        if row['OP'] == True:
            reduced_comment_df['OP'].append(int(1))
        else:
            reduced_comment_df['OP'].append(int(0))
        reduced_comment_df['Is Post'].append(int(0))
        reduced_comment_df['Post Date'].append(row['Post Date'])
        reduced_comment_df['Subreddit'].append(subreddit)
    reduced_comment_df = pd.DataFrame(reduced_comment_df)
    merge_dfs = pd.concat([merge_dfs,reduced_comment_df]).reset_index(drop=True)

# drop deleted comments/posts
merge_dfs = merge_dfs.drop(merge_dfs[merge_dfs['Text']=='[deleted]'].index)

# drop posts with no author
merge_dfs = merge_dfs.dropna(subset=['Author'])

# input indetifier for no text in post/comment
merge_dfs['Text'] = merge_dfs['Text'].fillna('[no text]')

# write to csv
merge_dfs.to_csv('../data/merged/mitch_master_data.csv',index=False)

In [306]:
merge_dfs.loc[(merge_dfs['ID'] == 'qb8k39')]

,ID,Title,Text,Author,OP,Is Post,Post Date,Subreddit
8980,qb8k39,Impulsive behavior triggered by feelings of in...,I'm just so messed up. I kinda need to rant ri...,DoktorVinter,1,1,1.634639e+09,BPDsupport
51515,qb8k39,Impulsive behavior triggered by feelings of in...,i know how you feel- i used to feel like no on...,ShyInu,0,0,1.635879e+09,BPDsupport
51516,qb8k39,Impulsive behavior triggered by feelings of in...,I no longer have any friends for over a year. ...,AffectionateMistake7,0,0,1.634691e+09,BPDsupport
51517,qb8k39,Impulsive behavior triggered by feelings of in...,Doesn't seem like a very lasting (or healthy) ...,DoktorVinter,1,0,1.634693e+09,BPDsupport
51518,qb8k39,Impulsive behavior triggered by feelings of in...,"I didn't chose this way of life, just the way ...",AffectionateMistake7,0,0,1.634693e+09,BPDsupport


In [307]:
merge_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51523 entries, 0 to 51522
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         51523 non-null  object 
 1   Title      51523 non-null  object 
 2   Text       50288 non-null  object 
 3   Author     49496 non-null  object 
 4   OP         51523 non-null  int64  
 5   Is Post    51523 non-null  int64  
 6   Post Date  51523 non-null  float64
 7   Subreddit  51523 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.1+ MB


In [318]:
merge_dfs.loc[merge_dfs['Text']=='[deleted]']

,ID,Title,Text,Author,OP,Is Post,Post Date,Subreddit


In [316]:
# drop deleted comments/posts
merge_dfs = merge_dfs.drop(merge_dfs[merge_dfs['Text']=='[deleted]'].index)

# drop posts with no author
merge_dfs = merge_dfs.dropna(subset=['Author'])

# input indetifier for no text in post/comment
merge_dfs['Text'] = merge_dfs['Text'].fillna('[no text]')


In [317]:
merge_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49496 entries, 0 to 51522
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         49496 non-null  object 
 1   Title      49496 non-null  object 
 2   Text       49496 non-null  object 
 3   Author     49496 non-null  object 
 4   OP         49496 non-null  int64  
 5   Is Post    49496 non-null  int64  
 6   Post Date  49496 non-null  float64
 7   Subreddit  49496 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.4+ MB
